In [29]:
# read data
import numpy as np
import pandas as pd
from collections import defaultdict
import re
import string 

text_file = open("crime.txt", "r")
lines = [i for i in text_file.readlines()]
data = list(lines)[1:]

                  

In [30]:
# hashtable like '92118': 8
# zipcode_crime_time is a dict : {'92092':'[list of times]'}
zipcode_CrimeNum = defaultdict(int)
zipcode_CrimeTime = defaultdict(list)

zipp = []
for d in data:
#     zipcode_numofcrime
    ele = d.split("\",\"")
    zipcode = ele[2]
    if (len(zipcode) != 5) or (zipcode[0] != '9'):
        continue
    zipcode_CrimeNum[zipcode] += 1
    if zipcode not in zipp:
        zipp.append(zipcode)
    
#     zipcode_CrimeTime
    crime_time = ""
    res = d.split(",")
    for crime_str in res:
        match = re.search('\d{1,2}\/\d{1,2}\/\d{4} \d{1,2}\:\d{1,2}\:\d{1,2}', crime_str)
        if(match):
            crime_time = crime_str
    zipcode_CrimeTime[zipcode] += [crime_time] 


In [31]:
for i in zipp:
    if(zipcode_CrimeNum[i] > 10):
        zipcode_CrimeNum[i] = int(zipcode_CrimeNum[i]/10)
    

In [35]:
from uszipcode import ZipcodeSearchEngine
search = ZipcodeSearchEngine()
xys = []
for i in zipp:
    zipcode123 = search.by_zipcode(i)
    if not zipcode123['Latitude']:
        continue
    mem = (zipcode123['Latitude'], zipcode123['Longitude'])
    for k in range(zipcode_CrimeNum[i]):
        xys.append(mem)


In [36]:
for tuple_pair in xys:
    assert tuple_pair and isinstance(tuple_pair, tuple)
    assert tuple_pair[0]
    assert isinstance(tuple_pair[0], float)
    assert tuple_pair[1]
    assert isinstance(tuple_pair[1], float)

In [37]:
from gmplot import gmplot

# Place map
gmap = gmplot.GoogleMapPlotter(32.7269669, -117.1647094, 13)

# Scatter points
top_attraction_lats, top_attraction_lons= zip(*xys)
#gmap.scatter(top_attraction_lats, top_attraction_lons, '#3B0B39', size=40, marker=False)

gmap.heatmap(top_attraction_lats, top_attraction_lons, threshold=20, radius=60, gradient=None, opacity=0.3, maxIntensity=200, dissipating=True)
# Draw
gmap.draw("my_map.html")
